In [ ]:
#########################
# FILE NAMES
#########################

#ORIGINAL FILE
# gender_dataset.csv

#NEW FILES
# INCREASED files are where I artifically increased the number of female artists.
# increased_test_gender_dataset.csv
# increased_train_gender_dataset.csv
# DECREASED files are where I artifically decreased the number of male artists.
# decreased_test_gender_dataset.csv
# decreased_train_gender_dataset.csv
import random

original_file = open("gender_dataset.csv", "r", encoding="utf-8")
original_lines = original_file.readlines()
original_file.close()

#INCREASED dataset
increased = open("increased_dataset.csv", "w+", encoding = "utf-8")
male = 0
female = 0
for line in original_lines:
    elements = line.split(",")
    if ("Male" in elements[-1]):
        male = male + 1
    if ("Female" in elements[-1]):
        female = female + 1
    increased.write(line)
num_women = female
while (num_women < male):
    for line in original_lines:
        elements = line.split(",")
        if (num_women < male):
            if ("Female" in elements[-1]):
                increased.write(line)
                num_women = num_women + 1
increased.close()

#DECREASED dataset
decreased = open("decreased_dataset.csv", "w+", encoding = "utf-8")
count_male = 0
count_female = 0
random.shuffle(original_lines)
for line in original_lines:
    elements = line.split(",")
    if ("Male" in elements[-1] and count_male <= female):
        count_male = count_male + 1
        decreased.write(line)
    if ("Female" in elements[-1] and count_female <= female):
        count_female = count_female + 1
        decreased.write(line)
decreased.close()

increased_file = open("increased_dataset.csv", "r", encoding="utf-8")
decreased_file = open("decreased_dataset.csv", "r", encoding="utf-8")
increased_lines = increased_file.readlines()
decreased_lines = decreased_file.readlines()

inc_test_gen = open("increased_test_gender_dataset.csv", "w+", encoding="utf-8")
inc_train_gen = open("increased_train_gender_dataset.csv", "w+", encoding="utf-8")

dec_test_gen = open("decreased_test_gender_dataset.csv", "w+", encoding="utf-8")
dec_train_gen = open("decreased_train_gender_dataset.csv", "w+", encoding="utf-8")

#takes 15% percent of data as test
counter = 0
for line in increased_lines:
    if (len(line) > 3):
        if (counter == 14):
            inc_test_gen.write(line)
            counter = 0;
        else:
            inc_train_gen.write(line)
            counter = counter + 1

counter = 0
for line in decreased_lines:
    if (len(line) > 3):
        if (counter == 14):
            dec_test_gen.write(line)
            counter = 0;
        else:
            dec_train_gen.write(line)
            counter = counter + 1

inc_test_gen.close()
inc_train_gen.close()
dec_test_gen.close()
dec_train_gen.close()
increased_file.close()
decreased_file.close()

In [ ]:
reset()

In [ ]:
import random

def make_test_set(percent, train_file, test_file, original_file):

    orig_file = open(original_file, "r", encoding="utf-8")
    lines = orig_file.readlines()
    
    trn_file = open(train_file, "w+", encoding = "utf-8")
    tst_file = open(test_file, "w+", encoding = "utf-8")

    counter = 0
    target = round((100/percent)) - 1
    for line in lines:
        if (len(line) > 3): #specific to MOMA dataset
            if (counter == target):
                tst_file.write(line)
                counter = 0;
            else:
                trn_file.write(line)
                counter = counter + 1
        
    trn_file.close()
    tst_file.close()
    orig_file.close()

def count_binary_output(original_file, output1, output2):
    original_file = open("gender_dataset.csv", "r", encoding="utf-8")
    original_lines = original_file.readlines()
    original_file.close()
    
    male = 0
    female = 0
    
    for line in original_lines:
        elements = line.split(",")
        if (output1 in elements[-1]):
            male = male + 1
        if (output2 in elements[-1]):
            female = female + 1
    
    return male, female

def oversample(orginal_file, percent_male, percent_female, new_file_name):
    male, female = count_binary_output(original_file, "Male", "Female")
    
    multiplier = percent_male / percent_female
    target_female_lines = round(male / multiplier)

    orig_file = open(original_file, "r", encoding="utf-8")
    lines = orig_file.readlines()
    orig_file.close()
    
    increased = open("temp.csv", "w+", encoding = "utf-8")
    train_set_name = new_file_name + "_train.csv"
    test_set_name = new_file_name + "_test.csv"
    
    increased_lines = []
    num_male = 0
    num_female = 0
    while (num_female < target_female_lines):
        for line in lines:
            elements = line.split(",")
            if ("Female" in elements[-1] and num_female < target_female_lines):
                increased_lines.append(line)
               # increased.write(line)
                num_female = num_female + 1
            elif ("Male" in elements[-1] and num_male < male):
                increased_lines.append(line)
                #increased.write(line)
                num_male = num_male + 1
    random.shuffle(increased_lines)
    #print(increased_lines)
    for il in increased_lines:
        increased.write(il)
    make_test_set(10, train_set_name, test_set_name, "temp.csv")
    increased.close()

def undersample(original_file, percent_male, percent_female, new_file_name):
        male, female = count_binary_output(original_file, "Male", "Female")
        
        multiplier = percent_male / percent_female
        target_male_lines = round(female * multiplier)
        
        orig_file = open(original_file, "r", encoding="utf-8")
        lines = orig_file.readlines()

        decreased = open("temp.csv", "w+", encoding = "utf-8")
        train_set_name = new_file_name + "_train.csv"
        test_set_name = new_file_name + "_test.csv"
        
        decreased_lines = []
        num_male = 0
        num_female = 0
        while (num_male < target_male_lines):
            for line in lines:
                elements = line.split(",")
                if ("Female" in elements[-1] and num_female < female):
                    decreased_lines.append(line)
                    num_female = num_female + 1
                elif ("Male" in elements[-1] and num_male < target_male_lines):
                    decreased_lines.append(line)
                    num_male = num_male + 1
        random.shuffle(decreased_lines)
        for dl in decreased_lines:
            decreased.write(dl)
        make_test_set(10, train_set_name, test_set_name, "temp.csv")
        decreased.close()

def make_ensemble(original_file, num_ensembles, new_file_name):
    orig_file = open(original_file, "r", encoding="utf-8")
    lines = orig_file.readlines()
    
    male, female = count_binary_output(original_file, "Male", "Female")
    target = male / num_ensembles
    
    male_lines = []
    female_lines = []
    
    for line in lines:
        elements = line.split(",")
        if ("Male" in elements[-1]):
            male_lines.append(line)
        if ("Female" in elements[-1]):
            female_lines.append(line)
    
    for ensemble in range(num_ensembles):
        nfn = new_file_name + str(ensemble+1)
        new_file = open(nfn, "w+", encoding="utf-8")
        
        ensembled_lines = []
        for fl in female_lines:
            ensembled_lines.append(fl)
        random.shuffle(male_lines)
        line_counter = 0
        for ml in male_lines:
            if line_counter < target:
                ensembled_lines.append(ml)
                male_lines.remove(ml)
            line_counter = line_counter + 1
        
        random.shuffle(ensembled_lines)
        for line in ensembled_lines:
            new_file.write(line)
        new_file.close()
        
        new_trn = nfn +"_train.csv"
        
        new_tst = nfn + "_test.csv"
        
        make_test_set(10, new_trn, new_tst, nfn)

original_file = "gender_dataset.csv"
#make test set for baseline model
make_test_set(10, "data/baseline/baseline_train.csv", "data/baseline/baseline_test.csv", original_file)
    
#make datasets for oversampling
oversample(original_file, .75, .25, "data/oversample/increased_2575")
oversample(original_file, .50, .50, "data/oversample/increased_5050")
oversample(original_file, .25, .75, "data/oversample/increased_7525")
    
#make datasets for undersampling
undersample(original_file, .75, .25, "data/undersample/decreased_2575")
undersample(original_file, .50, .50, "data/undersample/decreased_5050")
undersample(original_file, .25, .75, "data/undersample/decreased_7525")
    
#make datasets for ensemble
make_ensemble(original_file, 10, "data/ensemble/ensemble")

In [ ]:
original_file = open("data/ensemble/ensemble10_train.csv", "r", encoding="utf-8")
original_lines = original_file.readlines()
original_file.close()
file = open("data/ensemble/ensemble10_test.csv", "r", encoding="utf-8")
lines = file.readlines()
file.close()
male = 0
female = 0
for line in original_lines:
    elements = line.split(",")
    if ("Male" in elements[-1]):
        male = male + 1
    if ("Female" in elements[-1]):
        female = female + 1
for line in lines:
    elements = line.split(",")
    if ("Male" in elements[-1]):
        male = male + 1
    if ("Female" in elements[-1]):
        female = female + 1
        
total=male+female
print(total)
random = ((male/total)*(male/total))+((female/total)*(female/total))
print(random)